<a href="https://colab.research.google.com/github/kyleishaumbay/Umbay-Hierro-CSCI-161.03-UV2/blob/main/%5BUmbayHierro%5D_NLP_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load libraries

In [ ]:
%pip install scikit-learn
%pip install nltk
%pip install emoji

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng') # Download the specific English tagger

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

# Load data

In [ ]:
import pandas as pd

youtube_url = 'https://raw.githubusercontent.com/kyleishaumbay/Umbay-Hierro-CSCI-161.03-UV2/main/datasets/elonmusk_youtube_corpus.xlsx'

youtube_docs = pd.read_excel(youtube_url, index_col=0)

# Parse YouTube dates
youtube_docs['date_published'] = pd.to_datetime(
    youtube_docs['date_published'],
    utc=True,
    errors='coerce'
).dt.tz_convert('Asia/Manila').dt.tz_localize(None)

youtube_docs['source'] = 'youtube'
corpus = youtube_docs

# Sort by date
corpus = corpus.sort_values(by='date_published', ascending=False, na_position='last').reset_index(drop=True)

# Show result
corpus

,title,link,date_published,text,like_count,reply_parent_id,source
0,Taibbi is a journalist paid and bought by bill...,https://www.youtube.com/watch?v=BB__743Jfm0&lc...,2025-09-09 20:34:11,Taibbi is a journalist paid and bought by bill...,0,NaN,youtube
1,@@margobarranti Hello dear... I really appreci...,https://www.youtube.com/watch?v=kzlUyrccbos&lc...,2025-09-09 20:26:01,@@margobarranti Hello dear... I really appreci...,0,Ugy6M6SUBHCUtkU3RSh4AaABAg,youtube
2,Hello dear... I really appreciate your likes a...,https://www.youtube.com/watch?v=oO8w6XcXJUs&lc...,2025-09-09 20:05:41,Hello dear... I really appreciate your likes a...,0,UgzYT3Etk-P5otJtTKF4AaABAg,youtube
3,Hello dear... I really appreciate your likes a...,https://www.youtube.com/watch?v=oO8w6XcXJUs&lc...,2025-09-09 20:04:40,Hello dear... I really appreciate your likes a...,0,UgyIKvl9rpFtLHpw6Ul4AaABAg,youtube
4,Hello dear... I really appreciate your likes a...,https://www.youtube.com/watch?v=oO8w6XcXJUs&lc...,2025-09-09 20:04:28,Hello dear... I really appreciate your likes a...,0,UgyB14bghklGyJpGm2N4AaABAg,youtube
...,...,...,...,...,...,...,...
3197,&quot;In order for a person to make profit...&...,https://www.youtube.com/watch?v=5gnlhmaM-dM&lc...,2018-08-31 23:10:18,"""In order for a person to make profit..."" 17:0...",14,UgxF6E1eBVIDZIPSBQh4AaABAg,youtube
3198,Muskwatch,https://www.youtube.com/watch?v=5gnlhmaM-dM&lc...,2018-08-31 21:55:22,Muskwatch,21,UgxF6E1eBVIDZIPSBQh4AaABAg,youtube
3199,Skt skt skt skt...,https://www.youtube.com/watch?v=5gnlhmaM-dM&lc...,2018-08-31 21:46:16,Skt skt skt skt...,9,UgxF6E1eBVIDZIPSBQh4AaABAg,youtube
3200,&gt; 2 days ago<br><br>:thonk:,https://www.youtube.com/watch?v=5gnlhmaM-dM&lc...,2018-08-31 21:40:54,> 2 days ago\n\n:thonk:,33,UgxF6E1eBVIDZIPSBQh4AaABAg,youtube


# Preprocess text

## Load stopwords

In [ ]:
from pandas.errors import EmptyDataError
from nltk.corpus import stopwords

nltk.download('stopwords')

try:
  DOMAIN_STOPWORDS = list(
    pd.read_csv('https://raw.githubusercontent.com/kyleishaumbay/Umbay-Hierro-CSCI-161.03-UV2/main/datasets/domain_stopwords.txt', header=None).values.flatten()
  )
except EmptyDataError:
  DOMAIN_STOPWORDS = []

EN_STOPWORDS_LIST = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Clean corpus

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.corpus import wordnet
import emoji
import re
import numpy as np
import pandas as pd

def clean_corpus(corpus, text_column='text'):
    """
    Clean the text data in the specified column of the DataFrame.

    Steps:
    1. Convert to string & lowercase
    2. Remove URLs, mentions, emojis, punctuation, numbers
    3. Tokenize and clean
    4. POS tag and Lemmatize (nouns, verbs, adjectives, adverbs)
    5. Remove stopwords (general + domain)
    6. Final cleanup
    """
    cleaned_corpus = corpus.copy()

    # Ensure text column exists
    if text_column not in cleaned_corpus.columns:
        raise ValueError(f"Column '{text_column}' not found in DataFrame.")

    # Work on 'cleaned_text', keep original
    cleaned_corpus['cleaned_text'] = cleaned_corpus[text_column].astype(str)

    # === Early Cleaning: Remove noisy elements BEFORE tokenization ===
    # Remove URLs
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].str.replace(
        r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',
        '', regex=True
    )
    # Remove mentions (@username)
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].str.replace(
        r'(@[a-zA-Z0-9_]+)', '', regex=True
    )
    # Remove hashtags (optional — keep if you want to analyze them later)
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].str.replace(
        r'(#[a-zA-Z0-9_]+)', '', regex=True
    )
    # Remove emojis
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].apply(
        lambda x: emoji.replace_emoji(str(x), replace=' ')
    )
    # Remove punctuation and non-alphanumeric (except spaces)
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].str.replace(
        r'[^a-zA-Z\s]', ' ', regex=True
    )
    # Remove numbers
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].str.replace(
        r'\d+', ' ', regex=True
    )

    # Normalize whitespace (collapse multiple spaces)
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].str.replace(
        r'\s+', ' ', regex=True
    ).str.strip()

    # Handle NaN or empty strings
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].replace('', np.nan).fillna('')

    # === Now apply POS tagging, lemmatization, and stopwords ===
    lemmatizer = WordNetLemmatizer()

    # Function to convert NLTK POS tags to WordNet tags
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN # Default to noun if POS is not found


    def process_text(text):
        if not text or text == '':
            return ''
        # Tokenize
        tokens = text.split()
        # POS tag tokens
        tagged_tokens = pos_tag(tokens)
        # Lemmatize based on POS
        lemmatized_tokens = []
        for word, tag in tagged_tokens:
            wntag = get_wordnet_pos(tag)
            lemmatized_tokens.append(lemmatizer.lemmatize(word, pos=wntag))

        # Remove stopwords (general + domain)
        tokens_without_stopwords = [word for word in lemmatized_tokens if word not in EN_STOPWORDS_LIST]
        tokens_without_stopwords = [word for word in tokens_without_stopwords if word not in DOMAIN_STOPWORDS]

        # Rejoin
        return ' '.join(tokens_without_stopwords).strip()

    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].apply(process_text)

    # Final: remove any lingering empty or whitespace-only entries
    cleaned_corpus['cleaned_text'] = cleaned_corpus['cleaned_text'].replace('', np.nan)
    # Optionally drop rows where cleaned_text is NaN, or leave as empty
    # cleaned_corpus.dropna(subset=['cleaned_text'], inplace=True)

    return cleaned_corpus

In [ ]:
cleaned_corpus = clean_corpus(corpus, text_column='text')
cleaned_corpus

,title,link,date_published,text,like_count,reply_parent_id,source,cleaned_text
0,Taibbi is a journalist paid and bought by bill...,https://www.youtube.com/watch?v=BB__743Jfm0&lc...,2025-09-09 20:34:11,Taibbi is a journalist paid and bought by bill...,0,NaN,youtube,Taibbi journalist billionaire I SB Taibi usefu...
1,@@margobarranti Hello dear... I really appreci...,https://www.youtube.com/watch?v=kzlUyrccbos&lc...,2025-09-09 20:26:01,@@margobarranti Hello dear... I really appreci...,0,Ugy6M6SUBHCUtkU3RSh4AaABAg,youtube,Hello dear I appreciate comment I touch Send d...
2,Hello dear... I really appreciate your likes a...,https://www.youtube.com/watch?v=oO8w6XcXJUs&lc...,2025-09-09 20:05:41,Hello dear... I really appreciate your likes a...,0,UgzYT3Etk-P5otJtTKF4AaABAg,youtube,Hello dear I appreciate comment I touch Send d...
3,Hello dear... I really appreciate your likes a...,https://www.youtube.com/watch?v=oO8w6XcXJUs&lc...,2025-09-09 20:04:40,Hello dear... I really appreciate your likes a...,0,UgyIKvl9rpFtLHpw6Ul4AaABAg,youtube,Hello dear I appreciate comment I touch Send d...
4,Hello dear... I really appreciate your likes a...,https://www.youtube.com/watch?v=oO8w6XcXJUs&lc...,2025-09-09 20:04:28,Hello dear... I really appreciate your likes a...,0,UgyB14bghklGyJpGm2N4AaABAg,youtube,Hello dear I appreciate comment I touch Send d...
...,...,...,...,...,...,...,...,...
3197,&quot;In order for a person to make profit...&...,https://www.youtube.com/watch?v=5gnlhmaM-dM&lc...,2018-08-31 23:10:18,"""In order for a person to make profit..."" 17:0...",14,UgxF6E1eBVIDZIPSBQh4AaABAg,youtube,In order profit I probably miss two due cooper...
3198,Muskwatch,https://www.youtube.com/watch?v=5gnlhmaM-dM&lc...,2018-08-31 21:55:22,Muskwatch,21,UgxF6E1eBVIDZIPSBQh4AaABAg,youtube,Muskwatch
3199,Skt skt skt skt...,https://www.youtube.com/watch?v=5gnlhmaM-dM&lc...,2018-08-31 21:46:16,Skt skt skt skt...,9,UgxF6E1eBVIDZIPSBQh4AaABAg,youtube,Skt skt skt skt
3200,&gt; 2 days ago<br><br>:thonk:,https://www.youtube.com/watch?v=5gnlhmaM-dM&lc...,2018-08-31 21:40:54,> 2 days ago\n\n:thonk:,33,UgxF6E1eBVIDZIPSBQh4AaABAg,youtube,ago thonk


In [ ]:
import os

try:
  # Google Colab
  from google.colab import drive
  drive.mount('/content/drive')

  file_name = '/content/drive/My Drive/elonmusk_cleaned_corpus.xlsx'
except:
  # VS Code / Local machine
  file_name = 'elonmusk_cleaned_corpus.xlsx'

cleaned_corpus.to_excel(file_name)
print(f'File saved to {file_name}')

Mounted at /content/drive
File saved to /content/drive/My Drive/elonmusk_cleaned_corpus.xlsx


In [ ]:
# Export to CSV
csv_file_name = '/content/drive/My Drive/elonmusk_cleaned_corpus.csv'
cleaned_corpus.to_csv(csv_file_name, index=False) # index=False to avoid writing the DataFrame index as a column
print(f'File saved to {csv_file_name}')

File saved to /content/drive/My Drive/elonmusk_cleaned_corpus.csv


You can use the `files` module from `google.colab` to download the generated file directly to your local machine.

In [ ]:
from google.colab import files

# Save the DataFrame to a CSV file in the Colab environment first
colab_csv_file_name = 'elonmusk_cleaned_corpus.csv'
cleaned_corpus.to_csv(colab_csv_file_name, index=False)

# Download the file
files.download(colab_csv_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>